In [1]:
import networkx as nx         #pacchetto per reti
import numpy as np
import ipyparallel as ipp     #pacchetto per parallelizzazione
import time as clock
import matplotlib.pyplot as plt
from scipy import stats
import pickle
#import winsound
from Node import *            #pacchetto agente Node
from NetworkTool import *     #pacchetto rete
from GenerativeTool import *  #pacchetto generazione rete
from VoteTool import *        #pacchetto analisi voto
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def Core(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
     
    for i in range(Nodes):
        tfix=list(-2*np.random.random(FB)+1)
        tvar=list(-2*np.random.random(VB)+1)
        MG.add_node(Node(MG.number_of_nodes(),tfix,tvar, np.random.random(),TheTime)) #create nodes with FB fix belief and VB var beli
        
    TheTime=TheTime+1

    for i in range(MG.number_of_nodes()):
        tlay = np.random.randint(LayerC)
        MG.nodes()[i].aggF(TheTime)
        AttachChoosenBelief(MG, MG.nodes()[i], np.random.randint(RndFr)+1, tlay, fbpos[tlay], fbdist[tlay], vbpos[tlay], vbdist[tlay],TheTime) #every node search for friends
    print(' Time ', TheTime)

In [3]:
def Grow(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
    for j in range(Iter): #creating and adding
        TheTime = TheTime+1
        LayerB = ((LayerA-LayerC)//(Iter-1))*j + LayerC
        NodesB = NodesA//Iter
        print('Iter ',j,' Layers ', LayerB, ' Time ', TheTime)
        for i in range(NodesB):
            tfix=list(-2*np.random.random(FB)+1)
            tvar=list(-2*np.random.random(VB)+1)
            tnew = Node(MG.number_of_nodes(), tfix, tvar, np.random.random(), TheTime) #create nodes with FB fix belief and VB var belief
            MG.add_node(tnew) 
            tnew.aggF(TheTime)
            tlay = np.random.randint(LayerB)
            AttachChoosenBelief(MG, tnew, np.random.randint(RndFr)+1, tlay, fbpos[tlay], fbdist[tlay], vbpos[tlay], vbdist[tlay], TheTime)

    print(MG.number_of_nodes())
    print(' Time ', TheTime)

In [ ]:
def Rand(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime):
    TheTime=TheTime+1 
    co=0
    while MG.number_of_nodes()<NodesA:
        tfix=list(-2*np.random.random(FB)+1)
        tvar=list(-2*np.random.random(VB)+1)
        tnew = Node(MG.number_of_nodes(), tfix, tvar, np.random.random(), TheTime) #create nodes with FB fix belief and VB var belief
        MG.add_node(tnew) 
        tnew.aggF(TheTime)
        
    for i in MG.nodes():
        if len(MG.neighbors(i))==0:
            co=co+1
            RandomLink(MG, i, LayerA, TheTime)
    print("added: ",co,'; percentage: ', (co/MG.number_of_nodes())*100,'% ')
    print(' Time ', TheTime)
    return(co/MG.number_of_nodes())

In [ ]:

VoDAvg, VoDSkew, VaDAvg, VaDSkew, ChiAvg = {}, {}, {}, {}, {}

NodeList=[500,1000,1500,2000,2500]
FBlist=[5]
VBlist=[10]
LayerList=[4,6,8,12]
InfluDist=[0.2]

VoDAvg['proj'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
VoDSkew['proj'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
VaDAvg['proj'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
VaDSkew['proj'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
ChiAvg['proj'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
VoDAvg['alay'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
VoDSkew['alay'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
VaDAvg['alay'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
VaDSkew['alay'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}
ChiAvg['alay'] = {k:[] for k in NodeList+FBlist+VBlist+LayerList+InfluDist}

print(VoDAvg,'\n', VoDSkew,'\n', VaDAvg,'\n', VaDSkew,'\n', ChiAvg)
    
for rep in range(4):
    print('\n Repetition',rep)
    
    for NodesA in NodeList:
        for FB in FBlist:
            for VB in VBlist:
                for LayerA in LayerList:
                    for InDi in InfluDist:
                        
                        #yeah
                        print('\n Setup',NodesA,FB,VB,LayerA,InDi)
                        
                        MG = nx.MultiGraph()
                        TheTime = 0
                        RndFr=3
                        Nodes=50 #core set of 50 nodes
                        Iter=3
                        LayerC=2 #core layers
                        bd=0.05 #abstension strip
                        issue=np.random.random(FB+VB) #voting issue
                        Keys = [NodesA, FB, VB, LayerA, InDi]
                        print(' Time ', TheTime)
                        
                        TmpVDA=[]
                        TmpVDS=[]

                        fbdist, fbpos, vbdist, vbpos, cbpos = [], [], [], [], []
                        for j in range(LayerA): #creates random vector in specified range
                            fbpos.append(list(np.random.randint(FB, size=np.random.randint(1,FB))))
                            fbdist.append(0.50*np.random.random()+0.2)
                        for j in range(LayerA): #creates random vector in specified range
                            vbpos.append(list(np.random.randint(VB, size=np.random.randint(1,VB))))
                            vbdist.append(0.50*np.random.random()+0.2)
                            cbpos.append(list(np.random.randint(VB, size=np.random.randint(1,VB))))
                        print('\n',fbdist,'\n',fbpos,'\n',vbdist,'\n',vbpos,'\n',cbpos)

                        Core(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime)
                        Grow(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime)
                        print(Rand(MG, Nodes, NodesA, FB, VB, RndFr, fbpos, fbdist,vbpos, vbdist, LayerA, LayerC, TheTime))

                        print(TheTime)

                        #AllVoteA(MG, issue)

                        '''
                        print(FixDistPrnt(MG,0))
                        for n in range(LayerA):
                            G = ExtractLayer(MG, n)
                            print(FixDistPrnt(G,0))
                        '''
                        tt=VarDistPrnt(MG,0)
                        TmpVDA.append(tt[0])
                        TmpVDS.append(tt[2])
                        for n in range(LayerA):
                            G = ExtractLayer(MG, n)
                            tt=VarDistPrnt(G,0)
                            TmpVDA.append(tt[0])
                            TmpVDS.append(tt[2])
                            
                        BaseAnalisys(MG,LayerA)

                        EvolveInfluR(MG, 50000, cbpos, 0.75, InDi, TheTime, LayerA)
                        print(' Time ', TheTime)

                        AllVoteA(MG, issue)

                        tt=VoteDist(MG,0)
                        for key in Keys:
                            VoDAvg['proj'][key].append(tt[0])
                            VoDSkew['proj'][key].append(tt[2])
                        t0, t2 = [], []
                        for layer in range(LayerA):
                            G = ExtractLayer(MG, layer)
                            t0.append(VoteDist(G,0)[0])
                            t2.append(VoteDist(G,0)[2])
                        for key in Keys:
                            VoDAvg['alay'][key].append(np.mean(t0))
                            VoDSkew['alay'][key].append(np.mean(t2))
                            
                        tt=VarDistPrnt(MG,0)
                        for key in Keys:
                            VaDAvg['proj'][key].append(tt[0]-TmpVDA[0])
                            VaDSkew['proj'][key].append(tt[2]-TmpVDS[0])
                        t0, t2 = [], []
                        for n in range(LayerA):
                            G = ExtractLayer(MG, n)
                            t0.append(VarDistPrnt(G,0)[0])
                            t2.append(VarDistPrnt(G,0)[2])
                        for key in Keys:
                            VaDAvg['alay'][key].append(np.mean([i - j for i, j in zip(t0, TmpVDA[1:])]))
                            VaDSkew['alay'][key].append(np.mean([i - j for i, j in zip(t2, TmpVDS[1:])])) 

                        '''
                        print(FixDistPrnt(MG,1))
                        for n in range(LayerA):
                            G = ExtractLayer(MG, n)
                            print(FixDistPrnt(G,0))
                        '''   
                        tt=[]
                        for i in range(LayerA):
                            print('layer', i)
                            ChiA=ChiFreq(MG, 60, i) #return [Chi,Df, MGHist,GHist]
                            tt.append(ChiA[0]/ChiA[1])
                            #print('chi',ChiA[0],'df',ChiA[1],'chi rid',ChiA[0]/ChiA[1])
                            #plt.plot(ChiA[2])
                            #plt.plot(ChiA[3])
                            #plt.show
                        for key in Keys:
                            ChiAvg['alay'][key].append(np.mean(tt)) 
                                                
                        del tt, t0, t2
                        MG.clear()
                        print('\n', VoDAvg,'\n', VoDSkew,'\n', VaDAvg,'\n', VaDSkew,'\n', ChiAvg,'\n--------------------')
                        
                        
    with open('VoDAvg2.pickle', 'wb') as handle:
        pickle.dump(VoDAvg, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('VoDSkew2.pickle', 'wb') as handle:
        pickle.dump(VoDSkew, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('VaDAvg2.pickle', 'wb') as handle:
        pickle.dump(VaDAvg, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('VaDSkew2.pickle', 'wb') as handle:
        pickle.dump(VaDSkew, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('ChiAvg2.pickle', 'wb') as handle:
        pickle.dump(ChiAvg, handle, protocol=pickle.HIGHEST_PROTOCOL)

{'alay': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 4: [], 1500: []}, 'proj': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 4: [], 1500: []}} 
 {'alay': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 4: [], 1500: []}, 'proj': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 4: [], 1500: []}} 
 {'alay': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 4: [], 1500: []}, 'proj': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 4: [], 1500: []}} 
 {'alay': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 4: [], 1500: []}, 'proj': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [], 4: [], 1500: []}} 
 {'alay': {0.2: [], 2500: [], 5: [], 6: [], 8: [], 10: [], 12: [], 2000: [], 1000: [], 500: [

In [ ]:
print(VoDAvg,'\n', VoDSkew,'\n', VaDAvg,'\n', VaDSkew,'\n', ChiAvg)


In [ ]:
#VoteAnalysis(MG, issue, bd,1)
#VoteALayerLight(MG, issue, bd, 0, LayerA)

In [ ]:
#PollLayers(MG, 100, 100, bd, issue, 1, 2, 1)

In [ ]:
#PlotLayer(MG,LayerA)

In [ ]:
'''
VoDAvg, VoDSkew, VaDAvg, VaDSkew, ChiAvg
[proj] [alay]
NodeList=[500,1000,2000,3000]
FBlist=[5]
VBlist=[10]
LayerList=[4,8,12]
InfluDist=[0.2]
'''

In [ ]:
funname = ['VoDAvg', 'VoDSkew', 'VaDAvg', 'VaDSkew', 'ChiAvg']
dictionary = [VoDAvg, VoDSkew, VaDAvg, VaDSkew, ChiAvg]
#n=0
setting = ['alay','proj']
#m=1
varname = ['Nodes','Layers']
varlist = [NodeList, LayerList]
#o=1

for n in range(len(dictionary)):
    for m in range(len(setting)):
        for o in range(len(varlist)):

            temp0 = []
            temp1 = []
            temp2 = []
            temp3 = []
            print('\n', funname[n], setting[m], varname[o],'\n')

            for j in varlist[o]:
                #print(j)
                temp0.append(dictionary[n][setting[m]][j])
                temp1.append(np.nanmean(dictionary[n][setting[m]][j]))
                temp2.append(np.nanmax(dictionary[n][setting[m]][j]))
                temp3.append(np.nanmin(dictionary[n][setting[m]][j]))

                print('value',j,', average',np.nanmean(dictionary[n][setting[m]][j]),', st dev',np.nanstd(dictionary[n][setting[m]][j]))

            plt.plot(varlist[o],temp0, 'kx')
            plt.plot(varlist[o],temp1, 'r-')
            plt.plot(varlist[o],temp2, 'g-')
            plt.plot(varlist[o],temp3, 'g-')
            plt.ylabel(funname[n])
            plt.xlabel(varname[o])
            plt.savefig('CH2'+funname[n]+setting[m]+varname[o]+'.png',orientation='landscape',bbox_inches='tight',dpi='figure')
            plt.show()
